# Lesson 19 - Raw pointers

In [ ]:
#include <iostream>

## Variables have a location in memory.  Pointers store that.

In [ ]:
int a {5};

In [ ]:
std::cout << &a << std::endl;

References are just aliases to (another label for) the same location:

In [ ]:
int & a_ref {a};

In [ ]:
std::cout << &a_ref << std::endl;

Pointers are variables in their own right:

In [ ]:
int * a_ptr {&a};

...with their own location:

In [ ]:
std::cout << &a_ptr << std::endl;

...their value is the location of the thing they point to:

In [ ]:
std::cout << a_ptr << std::endl;

We can get the value they point to by de-referencing it:

In [ ]:
std::cout << *a_ptr << std::endl;

In [ ]:
std::cout << &(*a_ptr) << std::endl;

Unless initialising to a location, always initialise your pointers to the nullptr:

In [ ]:
int * null_ptr {nullptr};

Like with references you can do silly things if you are not careful:

In [ ]:
int * my_function() {
    int val {5};
    int * val_ptr {&val};
    std::cout << val_ptr << std::endl;
    std::cout << *val_ptr << std::endl;
    return val_ptr;
}

In [ ]:
int * my_ptr {my_function()};

In [ ]:
std::cout << my_ptr << std::endl;

In [ ]:
std::cout << *my_ptr << std::endl;

Pointers can point to pointers:

In [ ]:
int ** my_ptr_ptr {&a_ptr};

In [ ]:
std::cout << my_ptr_ptr << std::endl;

In [ ]:
std::cout << *my_ptr_ptr << std::endl;

In [ ]:
std::cout << **my_ptr_ptr << std::endl;

## Pointer arithmetic

In [ ]:
int an_int {0};
double a_double {0};

In [ ]:
int * an_int_ptr {&an_int};
double * a_double_ptr {&a_double};

In [ ]:
std::cout << an_int_ptr << std::endl;
std::cout << a_double_ptr << std::endl;

In [ ]:
std::cout << an_int_ptr+1 << std::endl;
std::cout << a_double_ptr+1 << std::endl;

In [ ]:
std::cout << sizeof(int) << std::endl;
std::cout << sizeof(double) << std::endl;

In [ ]:
++an_int_ptr;

In [ ]:
std::cout << an_int_ptr << std::endl;

**BEWARE** There are no checks to stop you changing a pointer pointing to memory you cannot access

## Pointers and objects

In [ ]:
class Base {
public:
    virtual void my_virtual_function() const {
        std::cout << "Base virtual function" << std::endl;
    }
    void my_function() const { std::cout << "Base function" << std::endl; }
};

In [ ]:
class Derived : public Base {
public:
    void my_virtual_function() const override {
        std::cout << "Derived virtual function" << std::endl;
    }
    void my_function() const { std::cout << "Derived function" << std::endl; }
}

In [ ]:
Derived derived_instance; // Uses default constructor
Derived * derived_ptr {&derived_instance};
Base * base_derived_ptr {&derived_instance};
Derived & derived_ref {derived_instance};
Base & base_derived_ref {derived_instance};

In [ ]:
derived_ptr->my_virtual_function();
derived_ptr->my_function();
derived_ref.my_virtual_function();
derived_ref.my_function();

In [ ]:
base_derived_ptr->my_virtual_function();
base_derived_ptr->my_function();
base_derived_ref.my_virtual_function();
base_derived_ref.my_function();

## C arrays

In [ ]:
int my_array[5];

In [ ]:
std::cout << my_array << std::endl;

In [ ]:
my_array[1] = 8;

In [ ]:
std::cout << my_array[1] << std::endl;
std::cout << *(my_array+1) << std::endl;

You can do the same with raw pointers:

In [ ]:
int some_int {8};
int * some_int_ptr {&some_int};

In [ ]:
std::cout << some_int_ptr << std::endl;

In [ ]:
std::cout << &some_int_ptr[1] << std::endl;

In [ ]:
std::cout << some_int_ptr+1 << std::endl;

**BEWARE** like with pointers, most compilers will not prevent you from trying to read beyond the end of the array (overflow) or the beginning (underflow).  This is the source of many security vulnerabilities in software.  You must be 100% sure of bounds when accessing C arrays, which is hard as they arrays themselves do not know their own size.  Usually this means adding extra bounds checks to your code.  Just use `std::array`, it does all this for you (and knows its own size)!

## Dynamic memory

### new/delete

In [ ]:
const int SIZE {5};

In [ ]:
int * my_new_function() {
    int * my_array {new int[SIZE]};
    for (int i {0}; i < SIZE; ++i) {
        my_array[i] = i;
    }
    return my_array;
}

In [ ]:
int * array_ptr {my_new_function()};

In [ ]:
std::cout << array_ptr[1] << std::endl;

In [ ]:
delete(array_ptr);

In [ ]:
std::cout << array_ptr[1] << std::endl;

In [ ]:
Derived * my_factory() {
    return new Derived(); // Note brackets to call constructor are REQUIRED!
}

In [ ]:
Derived * my_derived_factory_instance {my_factory()};

In [ ]:
my_derived_factory_instance->my_function();

In [ ]:
delete(my_derived_factory_instance);

### malloc/free

**BEWARE** This cannot be used with full classes (including those in the standard libarary) as the constructors cannot be called properly.

In [ ]:
char * make_string() {
    // We can do this directly with malloc because char is 1 byte
    char * my_string {static_cast<char *>(malloc(SIZE+1))};
    for(int i {0}; i < SIZE; ++i) {
        my_string[i] = '0' + i;
    }
    
    // Strings must be null-terminated -
    // this is what the extra malloc'd byte is for.
    my_string[SIZE] = '\0';

    return my_string;
}

In [ ]:
char * c_string {make_string()};

In [ ]:
std::cout << c_string << std::endl;

In [ ]:
std::string cpp_string {c_string};

In [ ]:
std::cout << cpp_string << std::endl;

In [ ]:
std::cout << cpp_string.length() << std::endl;

In [ ]:
free(c_string);

In [ ]:
int * my_ints {static_cast<int *>(malloc(sizeof(int) * SIZE))};

In [ ]:
for(int i {0}; i < SIZE; ++i) {
    my_ints[i] = i;
}

In [ ]:
for(int i {0}; i < SIZE; ++i) {
    std::cout << my_ints[i] << ", ";
}


In [ ]:
free(my_ints);

## Virtual destructors

In order for your classes to play nicely with `new`/`delete` and pointers you must make your destructors virtual to make sure all destructors are called.  **Always make your destructors virtual** so they will work no matter how your class is used (remember that std::vector, smart pointers etc. all use raw pointers and new/delete internally with your classes).

In [ ]:
class NonVirtualDestructor {
public:
    NonVirtualDestructor() {}
    ~NonVirtualDestructor() { std::cout << "NonVitualDestructor destroyed" << std::endl; }
}

In [ ]:
class VirtualDestructor {
public:
    VirtualDestructor() {}
    virtual ~VirtualDestructor() { std::cout << "VitualDestructor destroyed" << std::endl; }
}

In [ ]:
class DestructorDerived : public NonVirtualDestructor, public VirtualDestructor {
public:
    DestructorDerived() {}
    virtual ~DestructorDerived() { std::cout << "Derived destroyed" << std::endl; }
}

In [ ]:
VirtualDestructor * virtual_destructor_ptr {new DestructorDerived()};
NonVirtualDestructor * non_virtual_destructor_ptr {new DestructorDerived()};

In [ ]:
delete(virtual_destructor_ptr);

In [ ]:
delete(non_virtual_destructor_ptr);

## Variadic functions

In [ ]:
#include <stdarg.h> // the C header for variable length arguments

In [ ]:
void MyFunction(const int count, ...) {
    // Argument count must be the number of variable arguments
    va_list args; // argument information structure
    /*
     * Prepare to process the variable arguments -
     * first argument must be an instance of va_list, the second
     * is the last named argument (to figure out which arguments)
     * are in the variable list.
     */
    va_start(args, count);
    for(int i {0}; i < count; ++i) {
        /*
         * Get the next argument.  First argument to va_arg is the 
         * va_list structure passed to va_start, the second is the
         * type of the next argument.  Note that we have to code
         * the type for each argument here. va_arg will return that
         * type.
         */
        std::string next_arg {va_arg(args, char*)};
        std::cout << "Argument: " << next_arg << std::endl;
    }
    va_end(args); // Tell C(++) we have finished processing the arguments
}

In [ ]:
MyFunction(2, "Hello", "World");

In [ ]:
MyFunction(3, "Hi", "There");